In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import LambdaLR

# Define the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
# # Xavier initialization
#         torch.nn.init.xavier_uniform_(self.fc1.weight)
#         torch.nn.init.xavier_uniform_(self.fc2.weight)
# Normal distribution initialization
        torch.nn.init.normal_(self.fc1.weight)
        torch.nn.init.normal_(self.fc2.weight)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x



def to_one_hot(i, n):
    """Convert an integer to one-hot vector"""
    a = torch.zeros(n)
    a[i] = 1
    return a

def from_one_hot(a):
    """Convert a one-hot vector to an integer"""
    return torch.argmax(a).item()

def generate_dataset_one_hot(p, alpha, seed):
    """Generate a dataset with one-hot encoded inputs and outputs"""
    eye = torch.eye(p)
    dataset = []
    for a in range(p):
        for b in range(p):
            c = (a + b) % p
            ab_encoded = torch.cat((eye[a], eye[b]))
            c_encoded = to_one_hot(c, p)
            dataset.append((ab_encoded, c_encoded))
    dataset = [(ab.clone().detach(), c.clone().detach()) for ab, c in dataset]
    np.random.seed(seed) #Ensures reproducibility, can be changed by changing seed=42
    np.random.shuffle(dataset)
    train_set, test_set = train_test_split(dataset, train_size=alpha, shuffle=False)
    train_input, train_output = zip(*train_set)
    test_input, test_output = zip(*test_set)
    return list(train_input), list(train_output), list(test_input), list(test_output)

def decode_dataset(train_input, train_output, test_input, test_output):
    """Decode the dataset from one-hot encoded vectors to integers"""
    train_input_decoded = [(from_one_hot(a[:p]), from_one_hot(a[p:])) for a in train_input]
    train_output_decoded = [from_one_hot(c) for c in train_output]
    test_input_decoded = [(from_one_hot(a[:p]), from_one_hot(a[p:])) for a in test_input]
    test_output_decoded = [from_one_hot(c) for c in test_output]
    return train_input_decoded, train_output_decoded, test_input_decoded, test_output_decoded


def train_network(model, train_input, train_output, test_input, test_output, n_epochs, learning_rate):
   # Convert lists to tensors and move to device
    train_input = torch.stack(train_input).to(device)
    train_output = torch.stack(train_output).to(device)
    test_input = torch.stack(test_input).to(device)
    test_output = torch.stack(test_output).to(device)

    criterion = nn.CrossEntropyLoss()

    # Define the optimizer as AdamW
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), weight_decay=wd)


    scheduler = LambdaLR(optimizer, lambda step: min(step/100, 1))

    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        outputs_train = model(train_input)
        loss_train = criterion(outputs_train, train_output)
        loss_train.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            outputs_test = model(test_input)
            loss_test = criterion(outputs_test, test_output)

        train_losses.append(loss_train.item())
        test_losses.append(loss_test.item())

        predicted_train = torch.argmax(outputs_train.data, dim=1)
        predicted_test = torch.argmax(outputs_test.data, dim=1)
        train_output_indices = torch.argmax(train_output, dim=1)
        test_output_indices = torch.argmax(test_output, dim=1)
        train_correct = (predicted_train == train_output_indices).sum().item()
        test_correct = (predicted_test == test_output_indices).sum().item()
        train_acc = train_correct / len(train_output)
        test_acc = test_correct / len(test_output)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {loss_train.item():.8f}, Test Loss: {loss_test.item():.8f}, Train Acc: {train_acc*100:.2f}%, Test Acc: {test_acc*100:.2f}%')

    return train_losses, test_losses, train_accuracies, test_accuracies

Sat Aug  5 08:25:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
def create_model(p, hidden_dim=256):
    input_dim = 2 * p
    output_dim = p
    model = MLP(input_dim, hidden_dim, output_dim).to(device)  # Move the model to the device
    return model

def run_experiment(model, p, alpha, n_epochs, learning_rate, seed, train_network):
    # Generate the dataset with one-hot encoded outputs
    train_input, train_output, test_input, test_output = generate_dataset_one_hot(p, alpha, seed)

    train_losses, test_losses, train_accuracies, test_accuracies = train_network(
        model, train_input, train_output, test_input, test_output, n_epochs, learning_rate
    )

    return train_losses, test_losses, train_accuracies, test_accuracies

def plot_loss(train_losses, test_losses, p, seed, run):
    plt.figure(figsize=(4, 4))
    plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
    plt.plot(range(1, len(test_losses)+1), test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.yscale('log')  # Make y-axis logarithmic
    plt.legend()
    plt.title(f'Loss for p={p}, seed={seed}, run={run}')
    #plt.savefig(f"{images_dir}/loss_p_{p}_seed_{seed}_run_{run}.png")
    plt.close()

def plot_accuracy(train_accuracies, test_accuracies, p, seed, run):
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, len(test_accuracies)+1), test_accuracies, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend()
    plt.title(f'Accuracy for p={p}, seed={seed}, run={run}')
   # plt.savefig(f'{images_dir}/accuracy_p_{p}_seed_{seed}_run_{run}.png')
    plt.close()

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

images_dir = '/content/gdrive/My Drive/Images'


# Define parameters
#prime_numbers = [61, 89,97,113,131,157,193,239,269,277,307,331]
prime_numbers = [347, 367, 389,419,421] #More Primes
#prime_numbers = [967] #even More Primes

seeds = [42,56,12]
alpha = 0.6  # The ratio of the dataset to use for training
n_epochs = 700  # Set to a small number for testing. Increase to desired number for full training.
learning_rate = 0.022  # The learning rate for the optimizer
wd= 1

# Initialize dictionary to store results
results = {}

# Loop over prime numbers and seeds
for p in prime_numbers:
    for seed in seeds:

        for run in range(1, 3):
          # Create a new model for each prime number and seed
            model = create_model(p)
            print(f"Running experiment for p={p}, seed={seed}, run={run}...")
            # Run the experiment
            train_losses, test_losses, train_accuracies, test_accuracies = run_experiment(
                model, p, alpha, n_epochs, learning_rate, seed, train_network
            )
            # Store the results
            results[(p, seed, run)] = {
                "train_loss": train_losses,
                "test_loss": test_losses,
                "train_accuracy": train_accuracies,
                "test_accuracy": test_accuracies,
            }
            # Plot and save the loss and accuracy
            plot_loss(train_losses, test_losses, p, seed, run)
            plot_accuracy(train_accuracies, test_accuracies, p, seed, run)

# Print the keys of the results dictionary to check that the results have been stored correctly
print(results.keys())
torch.save(results, f'/content/gdrive/My Drive//Files/Research/Experiment/Prime_vs_time/Data/(Width 256)(More Primes) results_lr={learning_rate}_alpha={alpha}_wd={wd}.pt')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Running experiment for p=347, seed=42, run=1...
Epoch [100/700], Train Loss: 5.55632401, Test Loss: 6.14081049, Train Acc: 2.64%, Test Acc: 0.07%
Epoch [200/700], Train Loss: 5.06649494, Test Loss: 6.10437679, Train Acc: 9.95%, Test Acc: 0.07%
Epoch [300/700], Train Loss: 0.16031793, Test Loss: 0.47058153, Train Acc: 100.00%, Test Acc: 99.00%
Epoch [400/700], Train Loss: 0.04935727, Test Loss: 0.12693599, Train Acc: 100.00%, Test Acc: 99.99%
Epoch [500/700], Train Loss: 0.01980856, Test Loss: 0.05191702, Train Acc: 100.00%, Test Acc: 100.00%
Epoch [600/700], Train Loss: 0.00864764, Test Loss: 0.02370662, Train Acc: 100.00%, Test Acc: 100.00%
Epoch [700/700], Train Loss: 0.00388344, Test Loss: 0.01128101, Train Acc: 100.00%, Test Acc: 100.00%
Running experiment for p=347, seed=42, run=2...
Epoch [100/700], Train Loss: 5.56411314, Test Loss: 6.14011288, Train

In [ ]:
torch.save(results, f'/content/gdrive/My Drive//Files/Research/Experiment/Prime_vs_time/Data/results_lr={learning_rate}_alpha={alpha}_wd={wd}.pt')
